---
title: Feature engineering
description: "Understanding how to perform feature engineering"
format: html
toc: true
lang: en
jupyter: python3
ipynb-shell-interactivity: all
execute:
  echo: false
date: 2025-08-09 14:02 +0200
categories: ["datascience", "fundamentals"]
tags: ["notes", "featureengineering"]
comments:
  giscus:
    repo: jeev20/jeev20.github.io
---



# Feature engineering

This is my notes on feature engineering. The idea is to populate this article with all possible relevant knowledge I can get on my hands on and then use it as a reference in the future. Plan is also to use what I Study and note once, use many times.

```{mermaid}
mindmap
  root((Feature engineering))
    Feature transformation
        Handling missing values
            Imputation
                Remove observation
                Mean replacement
                Median replacement
                Most frequest categorical
        Handling categorical values
            One-hot-encoding
            Binning
        Handling outliers
            Outlier detection
            Outlier removal
        Feature scaling
            Standardization
            Normalization
    Feature construction
        Domain knowledge
        Experience
    Feature selection
        Feature importance 
    Feature extraction
```

In [ ]:
#| echo: true
import polars as pl
input_data_path = f"../data/iot/iot_telemetry_data.parquet"
df_original =  pl.read_parquet(input_data_path)
df_original.head(1)

## 1. Feature transformation

## Importing modules

In [ ]:
# | echo: true
import duckdb
import polars as pl
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

## Reading data

In [ ]:
# | echo: true
# ts is in epoch time format so converting it to timestamp
# rounding values for temperature and humidity
# converting temperature from farhaneit to celsius
df_raw = duckdb.sql(
    f"SELECT ts, to_timestamp(ts) AS timestamp, device, temp,ROUND((temp - 32) * 5.0 / 9, 4) AS temp_c, ROUND(humidity, 4) AS humidity, lpg, smoke, light FROM '{input_data_path}'"
)

## Exploring the data
The seven questions to get insight into the data 

### How big is the data? 

In [ ]:
# | echo: true
# Converting to polars to easy statistics and exploration
df_original = df_raw.pl()  
df_original.shape

### Imputation / Handling missing values

This dataset is quiet clean, there are no missing data in the input data in any feature. [Docs Reference](https://docs.pola.rs/user-guide/expressions/missing-data/)

In [ ]:
#| echo: true
df_original.null_count()

### Handling categorical variables

### Outlier detection

### Feature scaling


### Datetime transformation

The `ts` column is in unix seconds, which we will convert to timestamp.

In [ ]:
#| echo: true
df_original = df_original.with_columns(pl.from_epoch(pl.col("ts"), time_unit="s").alias("timestamp"))
df_original.head(2)

#### What is the date range of the data collected?

The time range of the dataset is

In [ ]:
#| echo: true
print(df_original.select(pl.col(['timestamp', 'ts'])).describe())

#### Find the average humidity level across all sensors.

In [ ]:
#| echo: true
df_original.select(pl.col("humidity").mean())

### Remove unwanted columns

We are only interested in certain columns, the rest of the columns can be removed.

In [ ]:
#| echo: true
irrelevant_columns = ["co", "lpg", "motion", "smoke", "ts", "light"]
for columnname in irrelevant_columns:
    pass
    #df = df_original.drop(columnname)
#df_original.head(1)

### Reorganize the columns

In [ ]:
#| echo: true
df = df_original.select(['timestamp', 'device', 'temp', 'humidity'])
df = df.sort(by="timestamp")
df.head(1)

In [ ]:
#| echo: true
import seaborn as sn
sn.lineplot(df.head(100), x="timestamp", y="temp")

In [ ]:
#| echo: true
from prophet import Prophet
import pandas as pd
import dyplot
df_device_1 = df.filter(pl.col("device")=="b8:27:eb:bf:9d:51")
df_device_2 = df.filter(pl.col("device")=="00:0f:00:70:91:0a")
df_device_3 = df.filter(pl.col("device")=="1c:bf:ce:15:ec:4d")


def get_forecast_data(df):
    device_df = df.select(pl.col(["timestamp", "temp"])).rename({"timestamp":"ds", "temp":"y"})
    device_df = device_df.to_pandas()
    m = Prophet()
    m.fit(device_df)

    future = m.make_future_dataframe(periods=4)
    forecast = m.predict(future)
    forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
    fig = m.plot(forecast)
    fig_comp = m.plot_components(forecast)
    #fig_dyplot = dyplot.prophet(m, forecast)
    #fig_dyplot.show()
    return fig, fig_comp

fig, fig_comp = get_forecast_data(df_device_1)
fig, fig_comp = get_forecast_data(df_device_2)
fig, fig_comp = get_forecast_data(df_device_3)